<a href="https://colab.research.google.com/github/lyeonsl/2025-Beyond-Summer-Project_library_seat_detection/blob/main/2025_Beyond_Summer_Project_Pre_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5주차(비대면) - 재라벨링한 Dataset Yolov8 학습
### 직접 수집한 도서관 이미지를 라벨링한 데이터셋으로 학습
>

In [ ]:
!nvidia-smi

### 런타임 다시 시작 & 캐시 초기화

In [1]:
import torch
torch.cuda.empty_cache()

### 0. zip 파일 업로드



In [ ]:
from google.colab import files
uploaded = files.upload()  # zip 파일 업로드

### 1. 압축 해제 및 경로 준비

In [4]:
import zipfile
import os

zip_path = "/content/Beyond.v3i.yolov8.zip"
extract_dir = "/content/Beyond.v3i.yolov8"

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

### 2. test/valid 나누기

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# 기존 데이터 경로
base_path = '/content/Beyond.v3i.yolov8'
img_dir = os.path.join(base_path, 'train/images')
lbl_dir = os.path.join(base_path, 'train/labels')

# 출력 경로
train_img_dir = os.path.join(base_path, 'images/train')
val_img_dir   = os.path.join(base_path, 'images/val')
train_lbl_dir = os.path.join(base_path, 'labels/train')
val_lbl_dir   = os.path.join(base_path, 'labels/val')

# 디렉토리 생성
for d in [train_img_dir, val_img_dir, train_lbl_dir, val_lbl_dir]:
    os.makedirs(d, exist_ok=True)

# 이미지 파일 리스트
img_files = [f for f in os.listdir(img_dir) if f.endswith(('.jpg', '.png'))]

# 학습/검증 분할
train_imgs, val_imgs = train_test_split(img_files, test_size=0.2, random_state=42)

def move_data(file_list, src_img_dir, src_lbl_dir, dst_img_dir, dst_lbl_dir):
    for f in file_list:
        img_src = os.path.join(src_img_dir, f)
        lbl_src = os.path.join(src_lbl_dir, os.path.splitext(f)[0] + '.txt')

        img_dst = os.path.join(dst_img_dir, f)
        lbl_dst = os.path.join(dst_lbl_dir, os.path.splitext(f)[0] + '.txt')

        shutil.copyfile(img_src, img_dst)
        if os.path.exists(lbl_src):
            shutil.copyfile(lbl_src, lbl_dst)

# 이동 실행
move_data(train_imgs, img_dir, lbl_dir, train_img_dir, train_lbl_dir)
move_data(val_imgs, img_dir, lbl_dir, val_img_dir, val_lbl_dir)

print("기존 이미지/라벨 → train/val 분할 완료")

In [ ]:
import os

# 디렉토리 경로 설정
base_dir = '/content/Beyond.v3i.yolov8/images'
subdirs = ['train', 'val']

# 이미지 확장자 정의 (YOLO에서 주로 사용되는 포맷들)
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

# 각 폴더에 있는 이미지 개수 출력
for subdir in subdirs:
    folder_path = os.path.join(base_dir, subdir)
    image_count = len([
        f for f in os.listdir(folder_path)
        if os.path.splitext(f)[1].lower() in image_extensions
    ])
    print(f"{subdir} 이미지 개수: {image_count}장")

### 3. 데이터 증강 및 데이터 합치기

In [ ]:
import os
import cv2
import random
import albumentations as A
from tqdm import tqdm

# 디렉토리 설정
input_img_dir = '/content/Beyond.v3i.yolov8/images/train'
input_lbl_dir = '/content/Beyond.v3i.yolov8/labels/train'

output_img_dir = '/content/augmented/images'
output_lbl_dir = '/content/augmented/labels'
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_lbl_dir, exist_ok=True)

# 클래스 수 (YOLO format assumes class indices)
class_names = ['empty', 'occupied']

# 증강 파이프라인 (bbox 처리 포함)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),
    A.GaussNoise(p=0.2),
    A.Resize(416, 416, p=1.0),  # YOLO 학습용 고정 크기
],
bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.3))

# 증강 횟수
AUG_PER_IMAGE = 2

image_files = [f for f in os.listdir(input_img_dir) if f.endswith(('.jpg', '.png'))]

for img_file in tqdm(image_files):
    img_path = os.path.join(input_img_dir, img_file)
    label_path = os.path.join(input_lbl_dir, os.path.splitext(img_file)[0] + '.txt')

    # 이미지 로드
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w = image.shape[:2]

    # 라벨 로드
    bboxes = []
    class_labels = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                cls, x, y, bw, bh = map(float, line.strip().split())
                bboxes.append([x, y, bw, bh])
                class_labels.append(int(cls))
    else:
        continue  # 라벨 없으면 스킵

    # 증강 반복
    for i in range(AUG_PER_IMAGE):
        augmented = augment(image=image, bboxes=bboxes, class_labels=class_labels)
        aug_img = augmented['image']
        aug_bboxes = augmented['bboxes']
        aug_labels = augmented['class_labels']

        # 파일명 설정
        base_name = os.path.splitext(img_file)[0]
        aug_img_name = f"{base_name}_aug{i}.jpg"
        aug_lbl_name = f"{base_name}_aug{i}.txt"

        # 이미지 저장
        aug_img_bgr = cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(output_img_dir, aug_img_name), aug_img_bgr)

        # 라벨 저장
        with open(os.path.join(output_lbl_dir, aug_lbl_name), 'w') as f:
            for label, box in zip(aug_labels, aug_bboxes):
                f.write(f"{label} {box[0]:.6f} {box[1]:.6f} {box[2]:.6f} {box[3]:.6f}\n")

In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# 증강된 이미지 및 라벨 경로
aug_img_dir = '/content/augmented/images'
aug_lbl_dir = '/content/augmented/labels'

# 클래스 이름 정의 (YOLO 라벨 인덱스 기준)
class_names = ['empty', 'occupied']

# 시각화할 이미지 수
NUM_SAMPLES = 5

# 이미지 파일 리스트
image_files = [f for f in os.listdir(aug_img_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 샘플 선택
for img_file in random.sample(image_files, min(NUM_SAMPLES, len(image_files))):
    img_path = os.path.join(aug_img_dir, img_file)
    label_path = os.path.join(aug_lbl_dir, os.path.splitext(img_file)[0] + '.txt')

    image = cv2.imread(img_path)
    h, w = image.shape[:2]

    if not os.path.exists(label_path):
        print(f"라벨 없음: {label_path}")
        continue

    # 라벨 읽기 및 바운딩박스 시각화
    with open(label_path, 'r') as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.strip().split())
            x1 = int((x - bw / 2) * w)
            y1 = int((y - bh / 2) * h)
            x2 = int((x + bw / 2) * w)
            y2 = int((y + bh / 2) * h)
            color = (0, 255, 0) if int(cls) == 0 else (0, 0, 255)
            label = class_names[int(cls)]
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    print(f"시각화: {img_file}")
    cv2_imshow(image)

In [ ]:
import os
import shutil

# 소스 디렉토리 목록
image_sources = [
    '/content/Beyond.v3i.yolov8/images/train',
    '/content/augmented/images'
]

label_sources = [
    '/content/Beyond.v3i.yolov8/labels/train',
    '/content/augmented/labels'
]

# 대상 디렉토리
merged_image_dir = '/content/merged/images'
merged_label_dir = '/content/merged/labels'

# 확장자 설정
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
label_extension = '.txt'

# 대상 디렉토리 생성
os.makedirs(merged_image_dir, exist_ok=True)
os.makedirs(merged_label_dir, exist_ok=True)

# 이미지 + 라벨 병합
for img_src, lbl_src in zip(image_sources, label_sources):
    for filename in os.listdir(img_src):
        if os.path.splitext(filename)[1].lower() in image_extensions:
            base_name, ext = os.path.splitext(filename)

            src_img_path = os.path.join(img_src, filename)
            src_lbl_path = os.path.join(lbl_src, f"{base_name}{label_extension}")

            dst_img_path = os.path.join(merged_image_dir, filename)
            dst_lbl_path = os.path.join(merged_label_dir, f"{base_name}{label_extension}")

            # 이름 중복 방지
            counter = 1
            while os.path.exists(dst_img_path) or os.path.exists(dst_lbl_path):
                new_base = f"{base_name}_{counter}"
                dst_img_path = os.path.join(merged_image_dir, f"{new_base}{ext}")
                dst_lbl_path = os.path.join(merged_label_dir, f"{new_base}{label_extension}")
                counter += 1

            shutil.copy2(src_img_path, dst_img_path)

            if os.path.exists(src_lbl_path):
                shutil.copy2(src_lbl_path, dst_lbl_path)

print("모든 이미지와 라벨이 /content/merged 디렉토리로 병합 완료됨")

In [ ]:
import os

# 경로 설정
image_dir = '/content/merged/images'
label_dir = '/content/merged/labels'

# 확장자 설정
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
label_extension = '.txt'

# 이미지 개수 세기
image_count = len([
    f for f in os.listdir(image_dir)
    if os.path.splitext(f)[1].lower() in image_extensions
])

# 라벨 개수 세기
label_count = len([
    f for f in os.listdir(label_dir)
    if f.endswith(label_extension)
])

print(f"이미지 수: {image_count}장")
print(f"라벨 수: {label_count}개")

In [ ]:
import yaml

class_names = ['empty', 'occupied']

data = {
    'train': '/content/merged/images',
    'val': '/content/Beyond.v3i.yolov8/images/val',
    'test': '/content/Beyond.v3i.yolov8/images/val',
    'names': {i: name for i, name in enumerate(class_names)},
    'nc': len(class_names)
}

yaml_path = '/content/merged/dataset.yaml'

with open(yaml_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print(f"dataset.yaml 파일 생성 완료: {yaml_path}")


### 4. YOLOv8 설치 및 모델 로드

In [ ]:
# ultralytics 설치
!pip install ultralytics

# 모델 로드
from ultralytics import YOLO
model = YOLO("yolov8n.pt")  # 경량 모델 (또는 yolov8s.pt 등 가능)


### 구글 코랩 테스트용

In [ ]:
# 모델 학습 시작
model.train(
    data=yaml_path,        # data.yaml 경로
    epochs=10,
    imgsz=416,
    batch=1,
    amp=True,
    cache=False,
    name="yolov8_chair"
)

In [ ]:
import glob

# 이미지 폴더 경로
image_folder = "/content/Beyond.v31.yolov8/merged/images"

# 이미지 확장자 확인
image_files = glob.glob(image_folder + "/*.jpg") + glob.glob(image_folder + "/*.png")

print(f"총 {len(image_files)}개의 이미지 파일이 있습니다.")
for f in image_files[:10]:  # 처음 10개만 출력
    print(f)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# 1. 모델 불러오기
model = YOLO('/content/runs/detect/yolov8_chair/weights/best.pt')  # 경로 확인 필요

# 2. 이미지 불러오기
image_path = ''
image = Image.open(image_path)

# 3. 예측 실행
results = model(image, conf=0.1)  # conf 낮춰보기

# 4. 결과 이미지 시각화
res_plotted = results[0].plot()  # 바운딩 박스가 그려진 이미지 (numpy 배열)

# 5. 이미지 출력
plt.imshow(res_plotted)
plt.axis('off')
plt.title("YOLOv8 Chair Detection Result")
plt.show()

# 6. 결과 출력 (박스가 실제로 있는지 확인)
print("탐지된 박스 수:", len(results[0].boxes))
print("박스 정보:", results[0].boxes.xyxy)
print("클래스 정보:", results[0].boxes.cls)
print("신뢰도:", results[0].boxes.conf)

In [ ]:
# 예측된 바운딩 박스 정보 출력
for box in results[0].boxes:
    cls = int(box.cls[0])
    conf = float(box.conf[0])
    xyxy = box.xyxy[0].tolist()
    print(f"Class: {model.names[cls]}, Conf: {conf:.2f}, BBox: {xyxy}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 경로: YOLO 학습 결과 디렉토리
results_csv_path = "/content/runs/detect/train2/results.csv"

# CSV 불러오기
assert os.path.exists(results_csv_path), f"CSV 파일 없음: {results_csv_path}"
df = pd.read_csv(results_csv_path)

# 기본 정보
epochs = df.index + 1  # 0부터 시작하므로 +1

# ✅ 그래프 그리기
plt.figure(figsize=(14, 6))

# ---- Accuracy / Precision / Recall ----
plt.subplot(1, 2, 1)
plt.plot(epochs, df["metrics/precision(B)"], label="Precision", marker='o')
plt.plot(epochs, df["metrics/recall(B)"], label="Recall", marker='o')
plt.plot(epochs, df["metrics/mAP50(B)"], label="mAP50", marker='o')
plt.title("Precision / Recall / mAP")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.grid(True)
plt.legend()

# ---- Loss ----
plt.subplot(1, 2, 2)
plt.plot(epochs, df["train/box_loss"], label="Box Loss", marker='x')
plt.plot(epochs, df["train/cls_loss"], label="Class Loss", marker='x')
plt.plot(epochs, df["val/box_loss"], label="Val Box Loss", linestyle="--")
plt.plot(epochs, df["val/cls_loss"], label="Val Class Loss", linestyle="--")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# 검출 실행 (validation 이미지 대상)
val_images_path = '/content/Beyond.v3i.yolov8/images/val'

results = model.predict(
    source=val_images_path,
    conf=0.3,
    save=True
)

# 시각화 결과 보기
results[0].show()

In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# YOLO val 결과 이미지 경로
val_result_dir = "/content/runs/detect/val"

# 파일 리스트 불러오기
img_files = [f for f in os.listdir(val_result_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 5장 선택
sample_imgs = random.sample(img_files, min(5, len(img_files)))

# 시각화
for f in sample_imgs:
    img_path = os.path.join(val_result_dir, f)
    img = cv2.imread(img_path)
    print(f"{f}")
    cv2_imshow(img)

### 5. 학습 실행 (960x960)

In [ ]:
# 모델 학습 시작
model.train(
    data=yaml_path,        # data.yaml 경로
    epochs=50,
    imgsz=960,
    batch=4,
    name="yolov8_chair"
)

In [ ]:
metrics = model.val()

# 주요 지표 출력
print(f"mAP@0.5:       {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

### 6. 학습된 모델로 예측 확인 (960x960)

In [ ]:
import glob

# 이미지 폴더 경로
image_folder = "/content/Beyond.v31.yolov8/merged/images"

# 이미지 확장자 확인
image_files = glob.glob(image_folder + "/*.jpg") + glob.glob(image_folder + "/*.png")

print(f"총 {len(image_files)}개의 이미지 파일이 있습니다.")
for f in image_files[:10]:  # 처음 10개만 출력
    print(f)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# 1. 모델 불러오기
model = YOLO('/content/runs/detect/yolov8_chair/weights/best.pt')  # 경로 확인 필요

# 2. 이미지 불러오기
image_path = ''
image = Image.open(image_path)

# 3. 예측 실행
results = model(image, conf=0.1)  # conf 낮춰보기

# 4. 결과 이미지 시각화
res_plotted = results[0].plot()  # 바운딩 박스가 그려진 이미지 (numpy 배열)

# 5. 이미지 출력
plt.imshow(res_plotted)
plt.axis('off')
plt.title("YOLOv8 Chair Detection Result")
plt.show()

# 6. 결과 출력 (박스가 실제로 있는지 확인)
print("탐지된 박스 수:", len(results[0].boxes))
print("박스 정보:", results[0].boxes.xyxy)
print("클래스 정보:", results[0].boxes.cls)
print("신뢰도:", results[0].boxes.conf)

In [ ]:
print(model.names)

In [ ]:
# 예측된 바운딩 박스 정보 출력
for box in results[0].boxes:
    cls = int(box.cls[0])
    conf = float(box.conf[0])
    xyxy = box.xyxy[0].tolist()
    print(f"Class: {model.names[cls]}, Conf: {conf:.2f}, BBox: {xyxy}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 경로: YOLO 학습 결과 디렉토리
results_csv_path = "/content/runs/detect/train2/results.csv"

# CSV 불러오기
assert os.path.exists(results_csv_path), f"CSV 파일 없음: {results_csv_path}"
df = pd.read_csv(results_csv_path)

# 기본 정보
epochs = df.index + 1  # 0부터 시작하므로 +1

# ✅ 그래프 그리기
plt.figure(figsize=(14, 6))

# ---- Accuracy / Precision / Recall ----
plt.subplot(1, 2, 1)
plt.plot(epochs, df["metrics/precision(B)"], label="Precision", marker='o')
plt.plot(epochs, df["metrics/recall(B)"], label="Recall", marker='o')
plt.plot(epochs, df["metrics/mAP50(B)"], label="mAP50", marker='o')
plt.title("Precision / Recall / mAP")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.grid(True)
plt.legend()

# ---- Loss ----
plt.subplot(1, 2, 2)
plt.plot(epochs, df["train/box_loss"], label="Box Loss", marker='x')
plt.plot(epochs, df["train/cls_loss"], label="Class Loss", marker='x')
plt.plot(epochs, df["val/box_loss"], label="Val Box Loss", linestyle="--")
plt.plot(epochs, df["val/cls_loss"], label="Val Class Loss", linestyle="--")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

### 7. Valid 세트 테스트 (960x960)

In [ ]:
# 검출 실행 (validation 이미지 대상)
val_images_path = '/content/Beyond.v3i.yolov8/images/val'

results = model.predict(
    source=val_images_path,
    conf=0.3,
    save=True
)

# 시각화 결과 보기
results[0].show()


In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# YOLO val 결과 이미지 경로
val_result_dir = "/content/runs/detect/val"

# 파일 리스트 불러오기
img_files = [f for f in os.listdir(val_result_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 5장 선택
sample_imgs = random.sample(img_files, min(5, len(img_files)))

# 시각화
for f in sample_imgs:
    img_path = os.path.join(val_result_dir, f)
    img = cv2.imread(img_path)
    print(f"{f}")
    cv2_imshow(img)

### 8. 학습 실행 (640x640)

In [ ]:
# 모델 학습 시작
model.train(
    data=yaml_path,        # data.yaml 경로
    epochs=50,
    imgsz=640,
    batch=1,
    amp=True,
    name="yolov8_chair"
)

Ultralytics 8.3.172 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_chair, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pre

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 18                  -1  1    123648  ultralytics.nn.modules.block.C2f             [192, 128, 1]                 
 19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
 20             [-1, 9]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 21                  -1  1    493056  ultralytics.nn.modules.block.C2f             [384, 256, 1]                 
 22        [15, 18, 21]  1    751702  ultralytics.nn.modules.head.Detect           [2, [64, 128, 256]]           
Model summary: 129 layers, 3,011,238 parameters, 3,011,222 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 46.3±24.6 MB/s, size: 97.5 KB)


train: Scanning /content/merged/labels... 558 images, 0 backgrounds, 0 corrupt: 100%|██████████| 558/558 [00:01<00:00, 355.24it/s]

train: New cache created: /content/merged/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 47.1±8.0 MB/s, size: 136.5 KB)


val: Scanning /content/Beyond.v3i.yolov8/labels/val... 47 images, 0 backgrounds, 0 corrupt: 100%|██████████| 47/47 [00:00<00:00, 140.56it/s]

val: New cache created: /content/Beyond.v3i.yolov8/labels/val.cache


Plotting labels to runs/detect/yolov8_chair/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/yolov8_chair
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/558 [00:00<?, ?it/s]

In [ ]:
metrics = model.val()

# 주요 지표 출력
print(f"mAP@0.5:       {metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95:   {metrics.box.map:.4f}")
print(f"Precision:      {metrics.box.mp:.4f}")
print(f"Recall:         {metrics.box.mr:.4f}")

### 9. 학습된 모델로 예측 확인 (640x640)

In [ ]:
import glob

# 이미지 폴더 경로
image_folder = "/content/Beyond.v31.yolov8/merged/images"

# 이미지 확장자 확인
image_files = glob.glob(image_folder + "/*.jpg") + glob.glob(image_folder + "/*.png")

print(f"총 {len(image_files)}개의 이미지 파일이 있습니다.")
for f in image_files[:10]:  # 처음 10개만 출력
    print(f)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import matplotlib.pyplot as plt

# 1. 모델 불러오기
model = YOLO('/content/runs/detect/yolov8_chair/weights/best.pt')  # 경로 확인 필요

# 2. 이미지 불러오기
image_path = ''
image = Image.open(image_path)

# 3. 예측 실행
results = model(image, conf=0.1)  # conf 낮춰보기

# 4. 결과 이미지 시각화
res_plotted = results[0].plot()  # 바운딩 박스가 그려진 이미지 (numpy 배열)

# 5. 이미지 출력
plt.imshow(res_plotted)
plt.axis('off')
plt.title("YOLOv8 Chair Detection Result")
plt.show()

# 6. 결과 출력 (박스가 실제로 있는지 확인)
print("탐지된 박스 수:", len(results[0].boxes))
print("박스 정보:", results[0].boxes.xyxy)
print("클래스 정보:", results[0].boxes.cls)
print("신뢰도:", results[0].boxes.conf)

In [ ]:
# 예측된 바운딩 박스 정보 출력
for box in results[0].boxes:
    cls = int(box.cls[0])
    conf = float(box.conf[0])
    xyxy = box.xyxy[0].tolist()
    print(f"Class: {model.names[cls]}, Conf: {conf:.2f}, BBox: {xyxy}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 경로: YOLO 학습 결과 디렉토리
results_csv_path = "/content/runs/detect/train2/results.csv"

# CSV 불러오기
assert os.path.exists(results_csv_path), f"CSV 파일 없음: {results_csv_path}"
df = pd.read_csv(results_csv_path)

# 기본 정보
epochs = df.index + 1  # 0부터 시작하므로 +1

# ✅ 그래프 그리기
plt.figure(figsize=(14, 6))

# ---- Accuracy / Precision / Recall ----
plt.subplot(1, 2, 1)
plt.plot(epochs, df["metrics/precision(B)"], label="Precision", marker='o')
plt.plot(epochs, df["metrics/recall(B)"], label="Recall", marker='o')
plt.plot(epochs, df["metrics/mAP50(B)"], label="mAP50", marker='o')
plt.title("Precision / Recall / mAP")
plt.xlabel("Epoch")
plt.ylabel("Metric Value")
plt.grid(True)
plt.legend()

# ---- Loss ----
plt.subplot(1, 2, 2)
plt.plot(epochs, df["train/box_loss"], label="Box Loss", marker='x')
plt.plot(epochs, df["train/cls_loss"], label="Class Loss", marker='x')
plt.plot(epochs, df["val/box_loss"], label="Val Box Loss", linestyle="--")
plt.plot(epochs, df["val/cls_loss"], label="Val Class Loss", linestyle="--")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

### 10. Valid 세트 테스트 (640x640)

In [ ]:
# 검출 실행 (validation 이미지 대상)
val_images_path = '/content/Beyond.v3i.yolov8/images/val'

results = model.predict(
    source=val_images_path,
    conf=0.3,
    save=True
)

# 시각화 결과 보기
results[0].show()

In [ ]:
import os
import random
import cv2
from google.colab.patches import cv2_imshow

# YOLO val 결과 이미지 경로
val_result_dir = "/content/runs/detect/val"

# 파일 리스트 불러오기
img_files = [f for f in os.listdir(val_result_dir) if f.endswith(('.jpg', '.png'))]

# 무작위 5장 선택
sample_imgs = random.sample(img_files, min(5, len(img_files)))

# 시각화
for f in sample_imgs:
    img_path = os.path.join(val_result_dir, f)
    img = cv2.imread(img_path)
    print(f"{f}")
    cv2_imshow(img)